In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install torch

^C
Note: you may need to restart the kernel to use updated packages.


     ---------------------------------------- 0.0/133.1 kB ? eta -:--:--
     ------------ -------------------------- 41.0/133.1 kB 1.9 MB/s eta 0:00:01
     --------------------------- ----------- 92.2/133.1 kB 1.3 MB/s eta 0:00:01
     ----------------------------------- -- 122.9/133.1 kB 1.4 MB/s eta 0:00:01
     -------------------------------------- 133.1/133.1 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/192.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/192.2 MB 6.4 MB/s eta 0:00:31
   ---------------------------------------- 0.2/192.2 MB 2.4 MB/s eta 0:01:22
   ---------------------------------------- 0.3/192.2 MB 2.4 MB/s eta 0:01:20
   ---------------------------------------- 0.4/192.2 MB 2.4 MB/s eta 0:01:22
   ---------------------------------------- 0.5/192.2 MB 3.0 MB/s eta 0:01:05
   ---------------------------------------- 0.5/192.2 MB 3.0 MB/s eta 0:01:05
   ---------------------------------------- 1.1/192.2 MB 4.0 MB/s et

In [1]:
import torch
from transformers import ViTFeatureExtractor, ViTForImageClassification
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score

c:\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load CIFAR-10 dataset from the datasets library
dataset = load_dataset("cifar10")

In [11]:
# Define the number of images you want to use for training and testing
num_train_images = 100
num_test_images = 50

# Select a subset of the dataset
train_dataset = dataset['train'].select(range(num_train_images))
test_dataset = dataset['test'].select(range(num_test_images))

In [12]:
# Define a feature extractor for ViT model
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")

In [13]:
# Preprocess the dataset
def preprocess_dataset(examples):
    inputs = feature_extractor(examples['img'], return_tensors='pt')
    return {**inputs, 'label': examples['label']}


In [14]:
train_dataset = train_dataset.map(preprocess_dataset)

Map: 100%|██████████| 100/100 [00:03<00:00, 27.72 examples/s]


In [15]:
test_dataset = test_dataset.map(preprocess_dataset)

Map: 100%|██████████| 50/50 [00:01<00:00, 28.44 examples/s]


In [16]:
# Define a ViT model for image classification
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224", num_labels=10)

c:\Python39\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\darkn\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./vit_cifar10",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    learning_rate=2e-4,
    save_steps=500,
    save_total_limit=3,
    push_to_hub=False,
)

In [ ]:
# Define a Trainer for training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# Train the model
trainer.train()


In [ ]:

# Evaluate the model
results = trainer.evaluate()


In [ ]:

# Calculate and print accuracy
accuracy = results['eval_accuracy']
print(f"Accuracy: {accuracy:.2f}")


In [ ]:

# Save the model
model.save_pretrained("./vit_cifar10_model")


In [ ]:

# Load the model
model = ViTForImageClassification.from_pretrained("./vit_cifar10_model")


In [ ]:

# Load test data and preprocess
test_data = dataset['test']['img']
test_input = feature_extractor(test_data, return_tensors='pt')


In [ ]:

# Make predictions
with torch.no_grad():
    logits = model(**test_input).logits


In [ ]:

# Convert logits to predictions
predictions = np.argmax(logits, axis=1)


In [ ]:

# Calculate accuracy
true_labels = dataset['test']['label']
accuracy = accuracy_score(true_labels, predictions)
print(f"Test Accuracy: {accuracy:.2f}")